<a href="https://colab.research.google.com/github/CodeCrafter-101/MINI_PROJECTS/blob/main/Email_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests-html lxml_html_clean

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.21.1 requires websockets<15.1.0,>=13.0.0, but you have websockets 10.4 which is inco

In [ ]:
from requests_html import AsyncHTMLSession
import re
import asyncio
import pandas as pd
from urllib.parse import urljoin, urlparse

#  Async scraping function
emails_set = set()
visited_urls = set()

async def scrape_emails_from_url(session, url, max_emails=20):
    global emails_set, visited_urls

    if len(emails_set) >= max_emails or url in visited_urls:
        return

    visited_urls.add(url)

    try:
        r = await session.get(url)
        await r.html.arender(timeout=20, sleep=1)

        text = r.html.text
        found_emails = set(re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', text))
        emails_set.update(found_emails)

        print(f"[+] {url} — {len(emails_set)} emails found")


        base_url = "{0.scheme}://{0.netloc}".format(urlparse(url))
        links = r.html.absolute_links
        internal_links = [link for link in links if link.startswith(base_url)]

        for link in internal_links:
            if len(emails_set) >= max_emails:
                break
            await scrape_emails_from_url(session, link, max_emails)

    except Exception as e:
        print(f"[!] Failed to fetch {url}: {e}")


async def main():
    session = AsyncHTMLSession()
    start_url = "https://www.thapar.edu"
    await scrape_emails_from_url(session, start_url, max_emails=20)


    emails_list = list(emails_set)[:20]
    df = pd.DataFrame(emails_list, columns=["Email"])
    df.to_csv("emails.csv", index=False)
    print("\n✅ Emails saved to 'emails.csv'")


await main()

[!] Failed to fetch https://www.thapar.edu: HTTPSConnectionPool(host='www.thapar.edu', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))

✅ Emails saved to 'emails.csv'
